# Одномерная минимизация криловлинейной psf.
* поиск дальней точки через кепстр
* начальное приближение - полный перебор на срединном перпендикуляре
* уточнение одним из (стандартных) алгоритмов
<img src="pic/minimization_ill.png">

Время работы: 6:24

In [ ]:
import numpy as np
from skimage import color, data, restoration, img_as_float, io
from skimage.restoration import richardson_lucy
from skimage.measure import compare_psnr

from scipy.signal import convolve2d
from scipy.optimize import minimize

from functools import partial
from tqdm import tqdm_notebook

from util import *
from richardson_lucy import *

In [ ]:
prefix = "one-dim-iterated"

In [ ]:
astro = color.rgb2gray(data.astronaut())
liftingbody = img_as_float(io.imread('liftingbody.png'))
iterations = 30

In [ ]:
sz = 21
deg = 5
points_real = [(deg-1)*sz//deg-1, sz//deg, sz-1, sz-1]
#points_real = [sz, 0, sz, sz]
psf_bezier = bezier_psf2(points_real)
plt.imshow(psf_bezier, 'gray')
plt.plot(0, 0, 'ro')
plt.plot(points_real[0], points_real[1], 'ro')
plt.plot(points_real[2], points_real[3], 'ro')
#plt.savefig("pic/{0}-psf.png".format(prefix))
plt.show()

In [ ]:
lifting_bezier_blurred = convolve2d(liftingbody, psf_bezier, 'same')
plt.imshow(lifting_bezier_blurred, 'gray')
plt.show()

In [ ]:
s_n, S_find = find_noise(lifting_bezier_blurred)
s_n

In [ ]:
N,M = lifting_bezier_blurred.shape
# ( ifft2 (100* log (1+ abs ( fft2 ( I ) ) ) ) ) ;
K = np.fft.ifftn(100*np.log(1+np.abs(np.fft.fftn(lifting_bezier_blurred))))
K_shift = np.fft.fftshift(K)

n, m, = K_shift.shape
c,r = np.unravel_index(np.argmin(K_shift, axis=None), K_shift.shape)
x0_2 = np.array([r - n//2, c - m//2])

if x0_2[0]<0:
    x0_2 = -x0_2
x0_1 = x0_2/2# + np.array([4,-4])
x0 = np.concatenate((x0_1, x0_2))
print(x0)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.clip(np.real(K_shift),-1,1), 'gray')#, vmin=noisy.min(), vmax=noisy.max())
plt.plot(n//2 + x0_2[1], m//2 + x0_2[0], 'ro')
plt.title('Кепстр изображения с выделенным минмиумом')
plt.show()

In [ ]:
x1_vec = np.array([x0_2[1], -x0_2[0]])
x1_vec = x1_vec / vec_len(x1_vec)
print(x1_vec)

In [ ]:
cost_fun = partial(funcToMinimizeCurvedNormal, xy2=x0_2, I_blurred=lifting_bezier_blurred, 
                   iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)

In [ ]:
L = vec_len(x0_2)
distances = np.linspace(-L/2, L/2, 30)
best_val = 1e9 #+inf
best_dist = 0
for dist in distances:
    val = cost_fun(dist)
    if val < best_val:
        best_dist = dist
        best_val = val

In [ ]:
best_dist, best_val

In [ ]:
point2 = second_point(x0_2, best_dist)
points_found = np.concatenate((point2, x0_2))
print(points_found)

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, 
                                     dampar=s_n, useFFT=False)
psnr_orig = compare_psnr_crop(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, 
                                     dampar=s_n, useFFT=False)
psnr_init = compare_psnr_crop(liftingbody, deconv_init['image'])

#points_found = res_my_grad2['x']
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, 
                                      dampar=s_n, useFFT=False)
psnr_found = compare_psnr_crop(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig.round(4), points_real), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init.round(4), x0.round(4)),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found.round(4), points_found.round(4))],
             savefig='pic/{0}.png'.format(prefix))

In [ ]:
x1_1 = point2
x1_2 = x0_2
for it in range(4):
    changed = False
    # move third point
    cost_fun1 = partial(funcToMinimizeCurved3, xy1=x1_1, I_blurred=lifting_bezier_blurred, 
                        iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
    res1 = minimize_grad(cost_fun1, x0=x1_2, alpha=100, maxiter=5)
    if not (x1_2 == res1.x).all():
        changed = True
    x1_2 = res1.x
    print(x1_1, x1_2)
    cost_fun2 = partial(funcToMinimizeCurved2, xy2=x1_1, I_blurred=lifting_bezier_blurred, 
                        iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
    res2 = minimize_grad(cost_fun2, x0=x1_1, alpha=100, maxiter=5)
    if not (x1_1 == res2.x).all():
        changed = True
    x1_1 = res2.x
    print(x1_1, x1_2)
    if not changed:
        break

In [ ]:
#deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, 
#                                     dampar=s_n, useFFT=False)
#psnr_orig = compare_psnr_crop(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(points_found)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, 
                                     dampar=s_n, useFFT=False)
psnr_init = compare_psnr_crop(liftingbody, deconv_init['image'])

points_found2 = np.concatenate((x1_1, x1_2))
psf_found = bezier_psf2(points_found2)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, 
                                      dampar=s_n, useFFT=False)
psnr_found = compare_psnr_crop(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig.round(4), points_real), 
                     'With one-dimensional search\nPSNR={0}\ncoord={1}'.format(psnr_init.round(4), points_found.round(4)),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found.round(4), points_found2.round(4))],
             savefig='pic/{0}-thiner.png'.format(prefix))

In [ ]:
plt.imshow(psf_drawn)
plt.plot(points_found2[0], points_found2[1], 'ro')
plt.plot(points_found[0], points_found[1], 'go')
plt.plot(points_found[2], points_found[3], 'go')
plt.show()

In [ ]:
plt.imshow(psf_found)
plt.show()